In [1]:
import os
import sys
from pathlib import Path

# lägg till projektroten (mappen ovanför notebooks/) på sys.path
root_dir = Path().absolute()

if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])

root_dir = str(root_dir) 
print(f"Root dir: {root_dir}")
print("Local environment")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Root dir: c:\Users\Chris\hockey-agent
Local environment
Added the following directory to the PYTHONPATH: c:\Users\Chris\hockey-agent


In [2]:
import hopsworks
from config import settings
import requests
import pandas as pd

c:\Users\Chris\anaconda3\envs\hockey-agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY,
    host = settings.HOPSWORKS_HOST
)


2025-12-16 11:54:32,828 INFO: Initializing external client
2025-12-16 11:54:32,828 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-16 11:54:34,261 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3193


In [4]:
import requests
import pandas as pd

STANDINGS_URL = "https://api-web.nhle.com/v1/standings/now"

resp = requests.get(STANDINGS_URL, timeout=20)
resp.raise_for_status()

standings_json = resp.json()

In [5]:
print(standings_json.keys())


dict_keys(['wildCardIndicator', 'standingsDateTimeUtc', 'standings'])


In [35]:
teams_raw = standings_json["standings"]
teams_df = pd.DataFrame(teams_raw)


In [13]:
print(teams_df.columns.tolist())


['conferenceAbbrev', 'conferenceHomeSequence', 'conferenceL10Sequence', 'conferenceName', 'conferenceRoadSequence', 'conferenceSequence', 'date', 'divisionAbbrev', 'divisionHomeSequence', 'divisionL10Sequence', 'divisionName', 'divisionRoadSequence', 'divisionSequence', 'gameTypeId', 'gamesPlayed', 'goalDifferential', 'goalDifferentialPctg', 'goalAgainst', 'goalFor', 'goalsForPctg', 'homeGamesPlayed', 'homeGoalDifferential', 'homeGoalsAgainst', 'homeGoalsFor', 'homeLosses', 'homeOtLosses', 'homePoints', 'homeRegulationPlusOtWins', 'homeRegulationWins', 'homeTies', 'homeWins', 'l10GamesPlayed', 'l10GoalDifferential', 'l10GoalsAgainst', 'l10GoalsFor', 'l10Losses', 'l10OtLosses', 'l10Points', 'l10RegulationPlusOtWins', 'l10RegulationWins', 'l10Ties', 'l10Wins', 'leagueHomeSequence', 'leagueL10Sequence', 'leagueRoadSequence', 'leagueSequence', 'losses', 'otLosses', 'placeName', 'pointPctg', 'points', 'regulationPlusOtWinPctg', 'regulationPlusOtWins', 'regulationWinPctg', 'regulationWins', 

In [36]:
teams_df["teamName"] = teams_df["teamName"].apply( # Don't need the french name for the team etc. 
    lambda x: x.get("default") if isinstance(x, dict) else x
)

teams_df = teams_df[[
    "teamName",
    "wins",
    "losses",
    "conferenceName",
    "divisionName",
    "winPctg",
    "points",
    "gamesPlayed",
    'goalAgainst', 
    'goalFor',
    'homeGamesPlayed',
    'homeLosses',
    'homeWins',
    'seasonId' 
]]

In [37]:
print(teams_df)

                 teamName  wins  losses conferenceName  divisionName  \
0      Colorado Avalanche    23       2        Western       Central   
1            Dallas Stars    22       7        Western       Central   
2     Carolina Hurricanes    21       9        Eastern  Metropolitan   
3          Minnesota Wild    19       9        Western       Central   
4    Vegas Golden Knights    16       6        Western       Pacific   
5      New York Islanders    19      11        Eastern  Metropolitan   
6           Anaheim Ducks    20      12        Western       Pacific   
7     Washington Capitals    18      10        Eastern  Metropolitan   
8     Tampa Bay Lightning    18      12        Eastern      Atlantic   
9       Detroit Red Wings    18      12        Eastern      Atlantic   
10    Philadelphia Flyers    16       9        Eastern  Metropolitan   
11     Montréal Canadiens    17      11        Eastern      Atlantic   
12          Boston Bruins    19      14        Eastern      Atla

In [38]:
import re

def to_snake(name: str) -> str:
    # splitta CamelCase till snake_case
    s = re.sub(r"(.)([A-Z][a-z]+)", r"\1_\2", name)
    s = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", s)
    return s.lower()


In [39]:
teams_df = teams_df.rename(columns={
    col: to_snake(col) for col in teams_df.columns
})
teams_df

,team_name,wins,losses,conference_name,division_name,win_pctg,points,games_played,goal_against,goal_for,home_games_played,home_losses,home_wins,season_id
0,Colorado Avalanche,23,2,Western,Central,0.718750,53,32,74,128,15,0,13,20252026
1,Dallas Stars,22,7,Western,Central,0.647059,49,34,90,115,17,5,11,20252026
2,Carolina Hurricanes,21,9,Eastern,Metropolitan,0.656250,44,32,90,108,17,5,11,20252026
3,Minnesota Wild,19,9,Western,Central,0.575758,43,33,87,99,18,3,11,20252026
4,Vegas Golden Knights,16,6,Western,Pacific,0.516129,41,31,90,96,15,4,7,20252026
5,New York Islanders,19,11,Eastern,Metropolitan,0.575758,41,33,93,100,18,6,10,20252026
6,Anaheim Ducks,20,12,Western,Pacific,0.606061,41,33,108,118,15,4,11,20252026
7,Washington Capitals,18,10,Eastern,Metropolitan,0.562500,40,32,83,106,17,5,10,20252026
8,Tampa Bay Lightning,18,12,Eastern,Atlantic,0.545455,39,33,88,106,16,8,8,20252026
9,Detroit Red Wings,18,12,Eastern,Atlantic,0.545455,39,33,109,104,16,6,9,20252026


In [41]:
fs = project.get_feature_store()

teams_fg = fs.get_or_create_feature_group(
    name="teams",
    description="NHL team information from standings endpoint",
    version=1,
    primary_key=["team_name", "season_id"]
)

In [42]:
teams_fg.insert(teams_df)

Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/fs/3140/fg/3143


Uploading Dataframe: 100.00% |██████████| Rows 32/32 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: teams_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3193/jobs/named/teams_1_offline_fg_materialization/executions


(Job('teams_1_offline_fg_materialization', 'SPARK'), None)